In [1]:
%load_ext tutormagic

# Function Example: Sounds
What's the point of higher-order functions? Here is an example that uses the concept of higher-order functions: **generating sound**.

## WAV Files
WAV is a standard format for encoding sound. It's not used much recently since it takes a lot of space. 

The Waveform Audio File Format encodes a sampled sound wave. 
<img src = 'wave.jpg' width = 400/>

Sampling allows us to digitize these sound waves by recording the height (amplitude) of the waves at particular moments (see the dots in the picture below)

<img src = 'amplitudes.jpg' width = 400/>

The dots indicates multiple samples at particular moment in time. For example, we can take `11,000` different samples every minute. 

Typically, these sound waves are generated by recording sound from real life. However, we can also generate waves from scratch according to mathematical functions. 

<img src = 'types.jpg' width = 500/>

The triangle wave is a simple wave form with the most pleasing sound. 

## Demo
We use some of the built-in modules to construct the wave files. 

In [32]:
from wave import open
from struct import Struct
from math import floor

The `frame_rate` is the sampling rate: "how many times per second do we want to have some values that indicates how high or how low the waveform of the sound.

In [33]:
frame_rate = 11025

The `encode` function is how we code a value according to what the `wave` file requires. This is just one way to obtain the value that we need.  

In [34]:
def encode(x):
    """Encode float x between -1 and 1 as two bytes.
    (See https://docs.python.org/3/library/struct.html)
    """
    i = int(16384 * x)
    return Struct('h').pack(i)

`play` is the function that writes the wave file. `play` takes a function `sampler` that describes the wave form that we are trying to generate. 

In [35]:
def play(sampler, name='song.wav', seconds=2):
    """Write the output of a sampler function as a wav file.
    (See https://docs.python.org/3/library/wave.html)
    """
    out = open(name, 'wb')
    out.setnchannels(1)
    out.setsampwidth(2)
    out.setframerate(frame_rate)
    t = 0
    while t < seconds * frame_rate:
        sample = sampler(t)
        out.writeframes(encode(sample))
        t = t + 1
    out.close()

And we have the function `tri` for generating a continuous triangle wave.

In [36]:
def tri(frequency, amplitude=0.3):
    """A continuous triangle wave."""
    period = frame_rate // frequency
    def sampler(t):
        saw_wave = t / period - floor(t / period + 0.5)
        tri_wave = 2 * abs(2 * saw_wave) - 1
        return amplitude * tri_wave
    return sampler

Below we have the frequency for the note `c`. We name it `c_freq`.

In [37]:
c_freq = 261.63

Now we're trying to create a triangle wave with the frequency of `c`.

In [38]:
c = tri(c_freq)
t = 0
while t < 100:
    print(c(t))
    t += 1

-0.3
-0.2714285714285714
-0.24285714285714285
-0.21428571428571427
-0.18571428571428572
-0.15714285714285714
-0.1285714285714286
-0.1
-0.07142857142857144
-0.04285714285714287
-0.014285714285714301
0.014285714285714301
0.04285714285714284
0.07142857142857144
0.09999999999999998
0.1285714285714286
0.1571428571428571
0.18571428571428572
0.21428571428571425
0.24285714285714285
0.2714285714285714
0.3
0.2714285714285714
0.24285714285714277
0.21428571428571433
0.18571428571428572
0.1571428571428571
0.1285714285714285
0.10000000000000005
0.07142857142857144
0.04285714285714284
0.014285714285714235
-0.014285714285714235
-0.04285714285714284
-0.07142857142857144
-0.10000000000000005
-0.1285714285714285
-0.1571428571428571
-0.18571428571428572
-0.21428571428571433
-0.24285714285714277
-0.2714285714285714
-0.3
-0.2714285714285715
-0.24285714285714277
-0.21428571428571433
-0.18571428571428558
-0.1571428571428571
-0.12857142857142864
-0.09999999999999991
-0.07142857142857144
-0.04285714285714297
-0

Above, we see the shape of the triangle wave. If we play `c`,

In [39]:
play(c)

Then Python will generate a playable `song.wav` file. 

This time, we'll use 3 different frequencies for 3 different notes,

In [40]:
c_freq, e_freq, g_freq = 261.63, 329.63, 392.00

And below is a function `both` that takes in 2 functions `f` and `g` and plays 2 frequencies on the same time. 

In [42]:
def both(f, g):
    return lambda t: f(t) + g(t)

Now we can construct a 2-note chord by playing both triangle wave on `c` and triangle wave on `e`.

In [44]:
play(both(tri(c_freq), tri(e_freq)))

Now the `song.wav` file has been overwritten. If we play it, notice that it sounds like a combination of 2 chords!

Now we're going to add a rhythm! Below we define a function `note` that takes a particular sound `f` and plays it from `start` to `end` (measured in seconds)

In [45]:
def note(f, start, end):
    def sampler(t):
        # Need to express t in seconds
        seconds = t / frame_rate
        # Checks if 'seconds' is between start and end
        if seconds < start:
            return 0
        elif seconds > end:
            return 0
        else: # Play the sound 
            return f(t)
    return sampler

In [46]:
c, e = tri(c_freq), tri(e_freq)

Below, Python creates a `song.wav` that plays the `c` chord from 0 to 0.25 seconds, and the `e` chord from 0.5 to 1 seconds.

In [47]:
play(both(note(c, 0 , 1/4), note(e, 1/2, 1)))

However, notice that the beginning sound and the ending sound sounds harsh. To solve this, we can add a `fade` to the `note` function.

In [48]:
def note(f, start, end, fade=.01):
    """Play f for a fixed duration."""
    def sampler(t):
        seconds = t / frame_rate
        if seconds < start:
            return 0
        elif seconds > end:
            return 0
        elif seconds < start + fade:
            return (seconds - start) / fade * f(t)
        elif seconds > end - fade:
            return (end - seconds) / fade * f(t)
        else:
            return f(t)
    return sampler

Below is a demonstration of playing the mario theme song,

In [49]:
def mario(c, e, g, low_g):
    z = 0
    song = note(e, z, z + 1/8)
    z += 1/8
    song = both(song, note(e, z, z + 1/8))
    z += 1/4
    song = both(song, note(e, z, z + 1/8))
    z += 1/4
    song = both(song, note(c, z, z + 1/8))
    z += 1/8
    song = both(song, note(e, z, z + 1/8))
    z += 1/4
    song = both(song, note(g, z, z + 1/4))
    z += 1/2
    song = both(song, note(low_g, z, z + 1/4))
    return song

def mario_at(octave):
    c = tri(octave * c_freq)
    e = tri(octave * e_freq)
    g = tri(octave * g_freq)
    low_g = tri(octave * g_freq / 2)
    return mario(c, e, g, low_g)

play(both(mario_at(1), mario_at(1/2)))